In [1]:
import pandas as pd
import numpy as np
import glob
import os
from tqdm import tqdm

In [2]:
globbed_files = glob.glob("user_profiles/*.csv") #creates a list of all csv files
globbed_files[:5]

['user_profiles\\user_profile_2022-02-01.csv',
 'user_profiles\\user_profile_2022-02-02.csv',
 'user_profiles\\user_profile_2022-02-03.csv',
 'user_profiles\\user_profile_2022-02-04.csv',
 'user_profiles\\user_profile_2022-02-05.csv']

In [3]:
data = [] # pd.concat takes a list of dataframes as an agrument
for csv in globbed_files:
    frame = pd.read_csv(csv)
    data.append(frame)


In [4]:
bigframe = pd.concat(data, ignore_index=True)
bigframe = bigframe.drop('reviews', axis=1)

In [5]:
bigframe.shape

(299667, 13)

In [6]:
bigframe.columns

Index(['user_id', 'user_name', 'about', 'stars', 'review_count',
       'member_since', 'contests_won', 'runner_up', '1_to_1_projects', 'tags',
       'repeat_clients', 'certification', 'timestamp'],
      dtype='object')

In [7]:
bigframe['level'] = bigframe['certification'].apply(lambda x: 'Entry' if 'Entry Level' in x else ('Mid' if 'Mid Level' in x else 'Top'))


In [8]:
# get unique solver
unique_solvers = bigframe.drop_duplicates(subset='user_id', keep="first")
unique_solvers.shape

(17365, 14)

In [9]:
ids = []
levels_start = []
levels_end = []
timestamps = []
for idx, row in tqdm(unique_solvers.iterrows()):
    id = row['user_id']
    level_start = row['level']
    ## find all info about that user
    solver_df = bigframe[(bigframe['user_id']==id) &(bigframe['level']!=level_start)]
    # print(solver_df)
    if not solver_df.empty:
        solver_df.reset_index(inplace=True)
        # print(solver_df)
        # print(id, level_start, solver_df['level'][0],  solver_df['timestamp'][0])
        ids.append(id); levels_start.append(level_start); levels_end.append(solver_df['level'][0])
        timestamps.append(solver_df['timestamp'][0])


524it [00:10, 47.76it/s]

In [ ]:
len(ids)

184

In [ ]:
user_level_change = pd.DataFrame({'user_id': ids, 'level_before': levels_start, 'level_after': levels_end, 'date_of_change': timestamps})

In [ ]:
user_level_change.to_csv('user_level_change.csv', index=False)

In [ ]:
user_level_change.groupby(['level_before', 'level_after'], as_index=False).count()

,level_before,level_after,user_id,date_of_change
0,Entry,Mid,70,70
1,Mid,Entry,84,84
2,Mid,Top,10,10
3,Top,Mid,20,20
